<a href="https://colab.research.google.com/github/JesseJames50/projeto_llm/blob/main/lang_chain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain-qdrant langchain-openai langchain

In [ ]:
!pip install python-dotenv

In [4]:
!pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 56.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 3.5 MB/s eta 0:00:00


In [13]:
# Importações necessárias
from langchain_qdrant import Qdrant
from langchain_community.vectorstores import Qdrant
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.embeddings import OpenAIEmbeddings
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams
from qdrant_client.http.models import PointStruct, VectorParams, Filter, FieldCondition, MatchValue
from datetime import datetime
import uuid
from dotenv import load_dotenv
import os

# Carrega as variáveis de ambiente do arquivo .env
load_dotenv()
apenai_key = os.getenv('OPENAI_API_KEY')
qdrant_key = os.getenv('QDRANT_API_KEY')



# Conecte-se ao Qdrant com autenticação
client_Qdrant = QdrantClient(
    url="https://20344d73-d460-4ad6-972f-7626b4af36bf.us-east4-0.gcp.cloud.qdrant.io:6333",
    api_key=qdrant_key
)

client = OpenAI(
  api_key= apenai_key,  # this is also the default, it can be omitted
)

# Nome da coleção
collection_name = 'openai_collection'

# Verificar se a coleção existe, caso contrário, criar
if not client_Qdrant.collection_exists(collection_name):
    client_Qdrant.create_collection(
        collection_name=collection_name,
        vectors_config=VectorParams(size=1536, distance=Distance.COSINE)
    )


# Verificar se a coleção já existe antes de criar
embeddings = OpenAIEmbeddings(api_key=apenai_key)

vectorstore = Qdrant(
        client=client_Qdrant,
        collection_name="openai_collection",
        embeddings=embeddings
    )

def get_chunks(text):
    text_splitter = CharacterTextSplitter(
        separator="\n",
        chunk_size=1000,
        chunk_overlap=200,
        length_function=len
    )
    chunks = text_splitter.split_text(text)
    return chunks

with open("./base_dados.txt") as f:
    raw_text = f.read()

texts = get_chunks(raw_text)

vectorstore.add_texts(texts)

qa = RetrievalQA.from_chain_type(
    llm=OpenAI(api_key= apenai_key),
    chain_type="stuff",
    retriever=vectorstore.as_retriever()
)
# Realizar uma consulta (por exemplo, "quantos por cento das solicitações são revisadas pelo comitê?")
query = "Como são realizadas as aprovações de compra e emissões de ordem de compra"
result = qa.run(query)

# Exibir os resultados
print(result)


 As aprovações de compra são realizadas após a confirmação da disponibilidade orçamentária pelo setor financeiro e as ordens de compra são emitidas após essa aprovação e encaminhadas ao fornecedor, com cópia para o solicitante e para o setor financeiro para registro e controle.
